In [1]:
import torch
from ucimlrepo import fetch_ucirepo 
from torch.utils.data import TensorDataset

from neural_blueprints.architectures import MLP
from neural_blueprints.config import MLPConfig, TrainerConfig
from neural_blueprints.utils import Trainer, accuracy
from neural_blueprints.preprocess import TabularPreprocessor

In [ ]:
data = fetch_ucirepo(id=2)
X = data.data.features
y = data.data.targets

In [ ]:
data = X.copy()
data['income'] = y

preprocessor = TabularPreprocessor(with_masking=False, normalize_discrete=True)
data, discrete_features, continuous_features = preprocessor.run(data)

X = data.drop(columns=['income'])
y = data['income'].values

dataset = TensorDataset(torch.tensor(X.values, dtype=torch.float32), torch.tensor(y.reshape(-1, 1), dtype=torch.float32))

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [4]:
# Define model configuration
mlp_config = MLPConfig(
    input_dim=X.shape[1],
    hidden_dims=[64, 32, 16],
    output_dim=1,
    normalization=None,
    activation='relu',
    final_activation=None
)

# Initialize model
model = MLP(mlp_config)
model.blueprint()

Sequential(
  (0): FeedForwardNetwork(
    (network): Sequential(
      (0): DenseLayer(
        (linear): Linear(in_features=14, out_features=64, bias=True)
        (normalization): Identity()
        (activation): ReLU()
      )
      (1): DenseLayer(
        (linear): Linear(in_features=64, out_features=32, bias=True)
        (normalization): Identity()
        (activation): ReLU()
      )
      (2): DenseLayer(
        (linear): Linear(in_features=32, out_features=16, bias=True)
        (normalization): Identity()
        (activation): ReLU()
      )
      (3): DenseLayer(
        (linear): Linear(in_features=16, out_features=1, bias=True)
        (normalization): Identity()
        (activation): Identity()
      )
    )
  )
  (1): Identity()
)


MLPConfig(input_dim=14, hidden_dims=[64, 32, 16], output_dim=1, normalization=None, activation='relu', final_activation=None)

In [6]:
trainer = Trainer(
    config=TrainerConfig(
        training_type="label",
        optimizer="adam",
        criterion="mse",
        learning_rate=0.001,
        weight_decay=1e-5,
        batch_size=32,
        save_weights_path="../models/mlp_adult.pth"
    ),
    model= model
)

# Train the model
trainer.train(train_dataset, val_dataset, epochs=5)

Directory ../models already exists. Weights file may be overwritten.


Training Epochs:  20%|██        | 1/5 [00:01<00:04,  1.05s/epoch]

Epoch 1/5, Training Loss: 0.0875, Validation Loss: 0.0804


Training Epochs:  40%|████      | 2/5 [00:02<00:03,  1.15s/epoch]

Epoch 2/5, Training Loss: 0.0761, Validation Loss: 0.0774


...

Epoch 3/5, Training Loss: 0.0738, Validation Loss: 0.0741


Training Epochs:  80%|████████  | 4/5 [00:04<00:01,  1.22s/epoch]

Epoch 4/5, Training Loss: 0.0730, Validation Loss: 0.0745


Training Epochs: 100%|██████████| 5/5 [00:05<00:00,  1.17s/epoch]


Epoch 5/5, Training Loss: 0.0727, Validation Loss: 0.0738


In [8]:
# Evaluate on test set
X_val = val_dataset[:][0]
y_val = val_dataset[:][1]

test_pred, test_loss = trainer.predict(X_val, y_val)
test_accuracy = accuracy(test_pred, y_val)
print(f"Val Loss: {test_loss:.4f}, Val Accuracy: {test_accuracy:.4f}")

Val Loss: 0.0738, Val Accuracy: 0.5768
